In [1]:
import pandas as pd
import numpy as np
import re
import datetime

# 1. Read Data

In [2]:
data = pd.read_csv('ODI-2019-csv.csv', sep=';')

# 2. Pre-Process Data

In [3]:
# Copy data dataframe to adapt
data_processed = data.copy()

## 2.1 Normalize Programmes

In [4]:
data_processed[data_processed.columns[1]] = data_processed[data_processed.columns[1]].str.lower()
data_processed[data_processed.columns[1]] = data_processed[data_processed.columns[1]].str.strip()

In [5]:
replacement = {'artificial intelligence':['artificial intelligence master','ai','m ai','master ai','msc artificial intelligence','msc artificial intelligence: cognitive science','uva ai'],
               'bioinformatics and systems biology':['bioinf','bioinformatics','bioinformatics & systems biology','bioinformatics and biology systems','bioinformatics and systems biology master\'s','bioinformatics and system biology','bioinformatics and systems biology msc','boinformatics','biosb','m bioinformatics','master of bioinformatics and system biology','msc bioinformatic','msc bioinformatics & systems biology','msc bioinformatics'],
               'computational science':['cls','clsjd','computational science joint degree','masters compuational science','msc computational science'],
               'computer science':['computer science (joint degree)','computer science masters','cs','cs (exchange)','master computer science','computer science - internet and web technologies', 'computer science big data engineering master','mscs','msc computer science (big data engineering)'],
               'business administration/business analytics':['mba','ba','business analytics and data science','business analytics','business administration','digital business & innovation','dbi digital business and innovation','master digital business and innovation', 'business administration: digital business & innovation','business adminitration','digital business and innovation'],
               'information sciences':['information studies: data science (uva)','ma data science (uva)','information studies','ds','uva: information systems - data science','uva msc information studies: data science','uva master information studies, track data science'],
               'finance':['quantitative risk management','qrm','msc qrm (dhp)','master of finance','dhpqrm','duisenberg qrm'],
               'humanities research':['rm humanities: linguistics','ma language technology'],
               'econometrics and operations research':['operation research','eor','econometrics and operations research','econometrics','financial econometrics','financial econometrivs','master econometrics','master econometrics & or','masters eor'],
               'health sciences':['master health sciences'],
               'management, policy analysis and entrepeneurship in health and life sciences':['master management, policy analysis and entrepeneurship in health and life sciences'],
               'other':['ms','x','multiple programmes'],
               'sociology':['exchange student (sociology)'],
               'stochastics and financial mathematics':['maths, stochastics, master, uva']}

In [6]:
for key in replacement:
    data_processed[data_processed.columns[1]]=data_processed[data_processed.columns[1]].replace(to_replace = replacement[key], value = key)

## 2.2 Normalize Birthdates

In [7]:
data_processed[data_processed.columns[8]] = data_processed[data_processed.columns[8]].str.lower()
data_processed[data_processed.columns[8]] = data_processed[data_processed.columns[8]].str.strip()

In [8]:
months = [(lambda x:datetime.date(1900, x, 1).strftime('%B'))(x) for x in range(13)[1:]]+([(lambda x:datetime.date(1900, x, 1).strftime('%b'))(x) for x in range(13)[1:]])
months = [x.lower() for x in months]

def monthfinder(date):
    for month in months:
        x = date.find(month)
        if x != -1:
            return month, x
    return None, None

In [9]:
def numericalsplit(date):
    #only day or month or year
    if len(date) <= 2:
        
        #only year
        if int(date) > 31:
            date = "//19" + date
            
        #assuming only month
        elif int(date) < 13:
            date = "/" + date + "/"
                
        #only day, contains no information
        else:
            date = "//" 
        
    #only day/month, month/year or year
    elif len(date) == 4:
        
        #only year
        if (int(date[:2]) == 19) & (int(date[2:4])>12):    
            date = "//" + date
        
        else:
            #assuming year/month
            if int(date[:2]) > 31:
                date = "/" + date[2:] + "/" + "19" + date[:2]
            
            #assuming month/year
            elif int(date[2:]) > 31:
                date = "/" + date[:2] + "/" + "19" + date[2:]
            
            #assuming month/day
            elif int(date[2:]) > 12:
                date = date[2:] + "/" + date[:2] + "/"
            
            #assuming day/month
            else:
                date = date[:2] + "/" + date[2:] + "/"
    
    #only day/month/year or month/year
    elif len(date) == 6:
        
        #assuming year/month
        if int(date[:2]) == 19 & (int(date[2:4])>12):
            date = "/" + date[4:] + "/" + date[:4]
        
        #assuming month/year
        elif int(date[2:4]) == 19 & (int(date[4:])>12):
            date = "/" + date[:2] + "/" + date[2:]
        
        #assuming month/day/year
        elif int(date[2:4]) > 12:
            date = date[2:4] + "/" + date[:2] + "/19" + date[4:]
        
        #assuming day/month/year
        else:
            date = date[:2] + "/" + date[2:4] + "/19" + date[4:]
    
    #day/month/year
    elif len(date) == 8:
        
        if (int(date[:2]) == 19) & (int(date[2:4])>12):
            
            #assuming year/month/day
            if int(date[6:]) > 12:
                date = date[6:] + "/" + date[4:6] + "/" + date[:4]
            
            #assuming year/day/month
            else:
                date = date[4:6] + "/" + date[6:] + "/" + date[:4]

        else:
            
            #assuming month/day/year
            if int(date[2:4]) > 12:
                date = date[2:4] + "/" + date[:2] + "/" + date[4:]
            
            #assuming day/month/year
            else:
                date = date[:2] + "/" + date[2:4] + "/" + date[4:]
    
    return date

In [10]:
def fixdate(date):
    datelist = date.split('/')
    
    datelist = list(filter(None, datelist))
    if len(datelist) == 0:
        date = "//"
        
    elif len(datelist) == 1:
        date = numericalsplit(datelist[0])

    elif len(datelist) == 2:
        
        # year/month
        if int(datelist[0]) > 31:

            # check if full year
            if len(datelist[0]) == 2:
                datelist[0] = "19" + datelist[0]
            
            date = "/".join(["",datelist[1],datelist[0]])
        
        # month/year
        elif int(datelist[1]) > 31:
            
            # check if full year
            if len(datelist[1]) == 2:
                datelist[1] = "19" + datelist[1]
                
            date = "/".join(["",datelist[0],datelist[1]])
            
        # month/day
        elif int(datelist[1]) > 12:
            date = "/".join([datelist[1],datelist[0],""])
        
        # day/month
        else:
            date = "/".join([datelist[0],datelist[1],""])
    
    else:

        if int(datelist[0]) > 31:

            # check if full year
            if len(datelist[0]) == 2:
                datelist[0] = "19" + datelist[0]

            # year/month/day
            if int(datelist[2]) > 12:
                date = "/".join([datelist[2],datelist[1],datelist[0]])
                
            # year/day/month
            else:
                date = "/".join([datelist[1],datelist[2],datelist[0]])

        # assuming never */year/*

        else:

            # check if full year
            if len(datelist[2]) == 2:
                datelist[2] = "19" + datelist[2]

            # month/day/year
            if int(datelist[1]) > 12:
                date = "/".join([datelist[1],datelist[0],datelist[2]])
            
            # day/month/year
            else:
                date = "/".join([datelist[0],datelist[1],datelist[2]])

    return date

In [11]:
def removesep(date, sep):
    datelist = []
    for x in date:
        if x == sep:
            pass
        else:
            datelist.append(x)
    date = "".join(datelist)
    return date

In [12]:
def replacesep(date):
    datelist = []
    for x in date:
        if not (x.isalpha() or x.isdigit()):
            datelist.append("/")
        else:
            datelist.append(x)
    date = "".join(datelist)
    date = re.sub('/+', '/', date)
    date = fixdate(date)
    return date

In [13]:
def replacemonth(date):
    month, x = monthfinder(date)
    while month != None:
        index = months.index(month)%12+1
        date = date.replace(month," " + str(index) + " ")
        month, x = monthfinder(date)
    if month == None:
        pass
    date = re.sub(' +', ' ', date)
    date = date.strip()
    return date

In [14]:
def removealpha(date):
    datelist = []
    for x in date:
        if x.isalpha():
            pass
        else:
            datelist.append(x)
    date = "".join(datelist)
    date = re.sub(' +', ' ', date)
    return date

In [15]:
dates = []
for date in data_processed[data_processed.columns[8]]:    
    date = removesep(date, "\'")
    
    if bool(re.match('^(?=.*/.*)',date)):
        date = replacesep(date)
    
    if removesep(date," ").isalpha():
        month, x = monthfinder(date)
        if month == None:                                    #contains no information
            date = "//"
        else:
            date = replacemonth(date)
            date = removealpha(date).strip()
            if date.isdigit():
                date = numericalsplit(date)
        
    else:
        date = replacemonth(date)
        date = removealpha(date)
        if date.isdigit():
            date = numericalsplit(date)
        else:
            date = replacesep(date)
    date = date.split('/')
    dates.append(date)

In [19]:
cols=['Birth Day','Birth Month','Birth Year']
data = pd.DataFrame(dates,columns=cols)
data = data.replace(r'', np.nan, regex=True)
data = data.astype('float64')
data.style.format({
    'A': '{:,f}'.format,
    'B': '{:,f}'.format,
})
#data.head(5)

,Birth Day,Birth Month,Birth Year
0,nan,nan,nan
1,9,7,1992
2,4,8,1995
3,29,9,1993
4,2,12,nan
5,31,12,1994
6,9,10,1995
7,22,7,1996
8,30,2,1945
9,nan,nan,nan


In [20]:
pd.concat([data_processed,data],axis=1)

,Timestamp,What programme are you in?,Have you taken a course on machine learning?,Have you taken a course on information retrieval?,Have you taken a course on statistics?,Have you taken a course on databases?,What is your gender?,Chocolate makes you.....,When is your birthday (date)?,Number of neighbors sitting around you?,Did you stand up?,"You can get 100 euros if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would deserve then?",Give a random number,Time you went to be Yesterday,What makes a good day for you (1)?,What makes a good day for you (2)?,What is your stress level (0-100)?,Birth Day,Birth Month,Birth Year
0,3/27/2019 10:16:11,artificial intelligence,yes,1,mu,ja,unknown,slim,unknown,3,no,100,6,23:00,sun,lecture,NaN,NaN,NaN,NaN
1,4/1/2019 15:58:21,artificial intelligence,no,0,mu,ja,male,fat,09071992,1,no,0,7,3,DM,AI,1,9.0,7.0,1992.0
2,4/1/2019 15:59:07,computer science,yes,1,unknown,ja,male,neither,04/08/95,0,unknown,0,0,0,0,0,100,4.0,8.0,1995.0
3,4/1/2019 15:59:30,econometrics and operations research,yes,1,mu,ja,male,fat,29-09-1993,2,no,42,"1,645","0,3",Winning,Food,0,29.0,9.0,1993.0
4,4/1/2019 15:59:58,computer science,no,1,mu,ja,male,neither,0212,2,no,second,66,20,kiss,love,100,2.0,12.0,NaN
5,4/1/2019 16:00:01,artificial intelligence,yes,1,mu,ja,male,unknown,31-12-1994,0,yes,Barkie,420,0000,Ganja,Sun,100,31.0,12.0,1994.0
6,4/1/2019 16:00:12,bioinformatics and systems biology,yes,0,mu,nee,female,neither,09-10-1995,Many,no,100,739,22:00,chocolate,Sun,80,9.0,10.0,1995.0
7,4/1/2019 16:00:16,bioinformatics and systems biology,yes,1,mu,ja,male,I have no idea what you are talking about,22/07/1996,0,yes,Don't so much,5,"24,3",Stay with my friends,Sun,50,22.0,7.0,1996.0
8,4/1/2019 16:00:21,computational science,no,0,mu,ja,male,neither,30-02-1945,0,no,0,31,03:30,Done something useful,Had fun,0,30.0,2.0,1945.0
9,4/1/2019 16:00:31,artificial intelligence,yes,1,mu,nee,male,neither,22,Infinite,no,Above average,5,02:00,A sunny day,Relax walks,80,NaN,NaN,NaN


## 2.3 Normalize Neighbors

In [21]:
def removenonnum(number):
    numlist = []
    for x in number:
        if not x.isdigit():
            if bool(re.match('^(?=.*,.*)',x)):
                numlist.append(x)
            else:
                pass
        else:
            numlist.append(x)
    number = "".join(numlist)
    number = re.sub(' +', ' ', number)
    return number

In [22]:
for index, row in data_processed.iterrows():
    neighbors = removenonnum(row[9])
    data_processed.iloc[index][9] = neighbors

data_processed[data_processed.columns[9]] = data_processed[data_processed.columns[9]].replace(r'', np.nan, regex=True)

## 2.4 Normalize DM Competition

In [25]:
for index, row in data_processed.iterrows():
    competition = removenonnum(row[11])
    data_processed.iloc[index][11] = competition

data_processed[data_processed.columns[9]] = data_processed[data_processed.columns[9]].replace(r'', np.nan, regex=True)

# 3. Save Pre-Processed Data

In [28]:
data_processed.to_csv('ODI-2019-processed.csv', header=True, index=False)